Travelling salesman problem

Developer: Tanmoy Das
Date: July 10, 2024

# Model Formulation and Pyomo implementation

In [19]:
# Library and data
import pyomo.environ as pyo
import numpy as np
from genetic_algorithm_tsp import GeneticAlgorithmTSP 
from tsp_model import TravelingSalesmanProblem
#!pip install glpk 

In [20]:
# Example Usage
num_cities = 10
tsp = TravelingSalesmanProblem(num_cities)
model = tsp.create_model()
route = tsp.solve()
tsp.print_route(route)

ERROR: evaluating object as numeric value: x[1,1]
        (object: <class 'pyomo.core.base.var.VarData'>)
    No value for uninitialized NumericValue object x[1,1]


ValueError: No value for uninitialized NumericValue object x[1,1]

In [16]:
model.pprint()
# model.constraint.pprint()

1 RangeSet Declarations
    N : Dimen=1, Size=10, Bounds=(1, 10)
        Key  : Finite : Members
        None :   True :  [1:10]

1 Param Declarations
    distance : Size=100, Index=N*N, Domain=Any, Default=None, Mutable=False
        Key      : Value
          (1, 1) :     0
          (1, 2) :    57
          (1, 3) :    74
          (1, 4) :    77
          (1, 5) :    77
          (1, 6) :    19
          (1, 7) :    93
          (1, 8) :    31
          (1, 9) :    46
         (1, 10) :    97
          (2, 1) :    80
          (2, 2) :     0
          (2, 3) :    98
          (2, 4) :    22
          (2, 5) :    68
          (2, 6) :    75
          (2, 7) :    49
          (2, 8) :    97
          (2, 9) :    56
         (2, 10) :    98
          (3, 1) :    91
          (3, 2) :    47
          (3, 3) :     0
          (3, 4) :    87
          (3, 5) :    82
          (3, 6) :    19
          (3, 7) :    30
          (3, 8) :    90
          (3, 9) :    79
         (3, 10) :    8

# Call Custom Genetic Algorithm for TSP

In [5]:
from genetic_algorithm_tsp import GeneticAlgorithmTSP 
import numpy as np
# !jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10

# Parameters for the GA
pop_size = 100
num_generations = 500
mutation_rate = 0.01

# data 
num_cities = 10
# Generate a random distance matrix for the GA
distance_matrix = np.random.randint(10, 100, size=(num_cities, num_cities))
np.fill_diagonal(distance_matrix, 0)  # Distance from a city to itself is 0

# Solve TSP using the Genetic Algorithm
ga_tsp_solver = GeneticAlgorithmTSP(distance_matrix, pop_size, num_generations, mutation_rate)
best_path, best_distance = ga_tsp_solver.evolve_population()

print("\nBest Path:", best_path)
print("Best Distance:", best_distance)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



-----
child, pointer [None, None, None, None, None, 4, 2, None, None, None] 7
-----
child, pointer [None, None, None, None, None, 4, 2, 0, None, None] 8
-----
child, pointer [None, None, None, None, None, 4, 2, 0, 5, None] 9
-----
child, pointer [None, None, None, None, None, 4, 2, 0, 5, 9] 0
-----
child, pointer [7, None, None, None, None, 4, 2, 0, 5, 9] 1
-----
child, pointer [7, 6, None, None, None, 4, 2, 0, 5, 9] 2
-----
child, pointer [7, 6, 8, None, None, 4, 2, 0, 5, 9] 3
-----
child, pointer [7, 6, 8, 1, None, 4, 2, 0, 5, 9] 4
-----
child, pointer [None, None, None, 9, 7, 6, 8, None, None, None] 7
-----
child, pointer [None, None, None, 9, 7, 6, 8, 3, None, None] 8
-----
child, pointer [None, None, None, 9, 7, 6, 8, 3, 5, None] 9
-----
child, pointer [None, None, None, 9, 7, 6, 8, 3, 5, 4] 0
-----
child, pointer [2, None, None, 9, 7, 6, 8, 3, 5, 4] 1
-----
child, pointer [2, 0, None, 9, 7, 6, 8, 3, 5, 4] 2
-----
child, pointer [None, 2, 3, 0, None, None, None, None, None, None] 

# Draw Network

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

# Define the number of cities and vehicles
num_cities = 10
num_vehicles = 2
depot = 0

# Generate a random distance matrix for the cities
np.random.seed(0)
distance_matrix = np.random.randint(10, 100, size=(num_cities, num_cities))
np.fill_diagonal(distance_matrix, 0)

# Define routes (assuming these are the routes obtained from a GA or any other method)
routes = [
    [0, 1, 4, 6, 0],  # Route for vehicle 1
    [0, 3, 7, 2, 5, 8, 0]  # Route for vehicle 2
]

# Create a graph
G = nx.DiGraph()

# Add nodes
for i in range(num_cities):
    G.add_node(i)

# Add edges with weights
for route in routes:
    for i in range(len(route) - 1):
        G.add_edge(route[i], route[i + 1], weight=distance_matrix[route[i], route[i + 1]])

# Get positions for all nodes
pos = nx.spring_layout(G)

# Draw the nodes
nx.draw_networkx_nodes(G, pos, node_size=700, node_color='lightblue')

# Draw the edges
for route in routes:
    edges = [(route[i], route[i + 1]) for i in range(len(route) - 1)]
    nx.draw_networkx_edges(G, pos, edgelist=edges, edge_color='black', arrows=True)

# Draw node labels
nx.draw_networkx_labels(G, pos, font_size=20, font_family='sans-serif')

# Draw edge labels
edge_labels = {(route[i], route[i + 1]): distance_matrix[route[i], route[i + 1]] for route in routes for i in range(len(route) - 1)}
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)

# Display the plot
plt.title("Vehicle Routing Network for TSP Problem")
plt.show()
